### 이 파일에서는 각 Class별로 crop_area.properties 파일에 있는 객체의 좌표를 참조하여 각 이미지들을 객체에 맞게 자릅니다.

# 1. Package Import

In [ ]:
from PIL import Image
import numpy
from osAdvanced import File_Control
import numpy as np
import os

# 2.  jpg 이외의 확장자들을 jpg로 변환시켜주기.

In [ ]:
# bmp gif GIF jpeg JPEG jpg JPG png PNG
extension_list = ['bmp', 'gif', 'GIF', 'jpeg', 'JPEG', 'JPG', 'png', 'PNG']
converted_data = 0
for ext in extension_list:
    data = File_Control.searchAllFilesInDirectory("/raid/korean_food_cropped", ext)
    for d in data:
        im = Image.open(d)
        rgb_im = im.convert('RGB')
        rgb_im.save(os.path.splitext(d)[0] + '.jpg')
        converted_data = converted_data + 1
print("Converted : ", converted_data)

# 3. 각 파일들의 Crop 좌표를 Dictionary 형태로 저장.
파일들을 불러온 뒤, 클래스별로 crop_area_properties에 있는 정보를 불러와서 해당 클래스에 속해있는 파일들의 Crop Coordinate 정보들을 Dictionary (파일이름을 Key, Crop Coordinate 정보를 value) 로 저장

In [ ]:
dataset = File_Control.searchAllFilesInDirectoryByDir("/raid/korean_food_cropped/", "jpg")
crop_area_dict = {}

for datas in dataset:
    crop_properties = datas[0].replace(os.path.basename(datas[0]), "crop_area.properties")
    base_path = datas[0].replace(os.path.basename(datas[0]), "")
    
    with open(crop_properties, 'rb') as f:
        crop_properties_byte = f.read()
    f.close()
    
    crop_area_list = crop_properties_byte.decode().split('\n')
    crop_area_list = crop_area_list[:len(crop_area_list)-1] #맨 마지막 줄은 항상 공백이므로 제거
    for crop_area in crop_area_list:
        t = crop_area.split('=')
        crop_area_dict[f"{base_path}{t[0]}.jpg"] = tuple([t for t in t[1].split(',')])

# 4. 결측치 제거
좌표가 제대로 적혀있지 않은 경우도 있었음. <br>
좌표가 제대로 적혀있지 않은 경우에는 Dictionary에서 제외시켜서 Crop대상이 아니도록 함.

In [ ]:
for key in list(crop_area_dict.keys()):
    if len(crop_area_dict[key]) != 4:
        crop_area_dict.pop(key)

# 5. Image Crop
위 코드에서 Dictionary 형태로 저장한 Crop Coordinate 정보를 불러와서 실제로 이미지를 Crop 함.

In [ ]:
cropped_imgs = {}
for data in list(crop_area_dict.keys()):
    im = Image.open(data)
    x = int(crop_area_dict[data][0])
    y = int(crop_area_dict[data][1])
    w = int(crop_area_dict[data][2])
    h = int(crop_area_dict[data][3])
    cropped_imgs[data] = im.crop((x, y, x+w, y+h))
    

In [ ]:
cropped_imgs

# 6. 실제로 Crop이 제대로 잘 되었는지 확인함.
Crop 된 이미지를 랜덤으로 불러와서 확인.

In [ ]:
import random 
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
prop = fm.FontProperties(fname=path, size=18)

columns = 4
rows = 2
fig = plt.figure(figsize=(20, 10))
ax = []
for i in range(columns*rows):
    img_index = random.randint(0, len(cropped_imgs))
    cropped_img = np.array(list(cropped_imgs.values())[img_index])
    ax.append(fig.add_subplot(rows, columns, i+1))
    ax[-1].set_title(list(cropped_imgs.keys())[img_index], fontproperties=prop)
    plt.imshow(cropped_img)
plt.show()

# 7. Crop된 이미지를 저장함.

In [ ]:
for cropped_img in cropped_imgs:
    cropped_imgs[cropped_img].convert("RGB").save(cropped_img)
    print("saved : " + cropped_img)